In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [2]:
# read cases_by_county.csv
cc_df = pd.read_csv('cases_by_county.csv',index_col=0)
cc_df['date'] = pd.to_datetime(cc_df['date'],format='%Y-%m-%d')
# read wastewater_by_county.csv
wc_df = pd.read_csv('wastewater_by_county.csv',index_col=0)
wc_df['sampling_week'] = pd.to_datetime(wc_df['sampling_week'],format='%Y-%m-%d')
# read NWSS_Concentration_Data.csv
cd_df = pd.read_csv('NWSS_Concentration_Data.csv')
cd_df['date'] = pd.to_datetime(cd_df['date'],format='%Y-%m-%d')
# read NWSS_Metric_Data.csv
md_df = pd.read_csv('NWSS_Metric_Data.csv',engine='python')
md_df['date_start'] = pd.to_datetime(md_df['date_start'],format='%Y-%m-%d')
md_df['date_end'] = pd.to_datetime(md_df['date_end'],format='%Y-%m-%d')
md_df['first_sample_date'] = pd.to_datetime(md_df['first_sample_date'],format='%Y-%m-%d')
# construct new dataframe
df = pd.DataFrame(columns=['wwtp_id','county_fips','county','state','population_served','ww_date','flow_rate','per_capita_flow_rate','sample_location','pcr_conc_gc/cap/d','normalization_method','sampling_week','pcr_conc_gc/mL','case_date','cases_per_100k','cases'])

In [3]:
md_wwtp_id_arr = md_df['wwtp_id'].unique()
md_county_fips_arr = md_df['county_fips'].unique()
cc_county_fips_arr = cc_df['fipscode'].unique()
wc_county_fips_arr = wc_df['fipscode'].unique()
county_fips_arr = np.intersect1d(cc_county_fips_arr,wc_county_fips_arr)

# Confirm all WWTP in md_df match the WWTP in cd_df
cd_wwtp_id_li = []
for ind,val in cd_df['key_plot_id'].items():
    if val.split('_')[0] == 'CDC':
        v = int(val.split('_')[3])
        if v not in cd_wwtp_id_li:
            cd_wwtp_id_li.append(v)
    elif val.split('_')[0] in ['NWSS','WWS']:
        v = int(val.split('_')[2])
        if v not in cd_wwtp_id_li:
            cd_wwtp_id_li.append(v)
cd_wwtp_id_arr = np.array(cd_wwtp_id_li)
np.intersect1d(md_wwtp_id_arr,cd_wwtp_id_arr).shape

(1249,)

In [4]:
# filter out the wwtps from the county_fips_arr for this study
county_df = pd.DataFrame(columns=['county_fips','wwtp_id','county','state'])
wwtp_id_li = []
for i in county_fips_arr:
    if i<10000:
        i_str = '0'+str(i)
    else:
        i_str = str(i)
    wwtp_li = list(md_df[md_df['county_fips']==i_str]['wwtp_id'].unique())
    if len(wwtp_li) > 0:
        wwtp_id_li.extend(wwtp_li)
        county = md_df[md_df['county_fips']==i_str]['county_names'].unique()[0]
        state = md_df[md_df['county_fips']==i_str]['wwtp_jurisdiction'].unique()[0]
        row_to_append = pd.DataFrame({'county_fips':[i_str],'wwtp_id':[wwtp_li],'county':county,'state':state}) #
        county_df = pd.concat([county_df,row_to_append])
county_df = county_df.reset_index()
county_df = county_df.drop('index',axis=1)
for ind,val in county_df.iterrows():
    pop_li = []
    sample_location_li = []
    for wwtp_id in val['wwtp_id']:
        pop_li.append(md_df[md_df['wwtp_id'] == wwtp_id]['population_served'].unique()[0])
        sample_location_li.append(md_df[md_df['wwtp_id'] == wwtp_id]['sample_location'].unique()[0])
    county_df.loc[ind,['pop_li','sample_location_li']] = str(pop_li), str(sample_location_li)
print(len(wwtp_id_li))
print(county_df)

364
    county_fips           wwtp_id      county      state  \
0         01033            [1333]     Colbert    Alabama   
1         01081            [1464]         Lee    Alabama   
2         01117            [1394]      Shelby    Alabama   
3         02110      [1490, 1491]      Juneau     Alaska   
4         04012             [501]      La Paz    Arizona   
..          ...               ...         ...        ...   
180       55025  [391, 1277, 231]        Dane  Wisconsin   
181       55073             [433]    Marathon  Wisconsin   
182       56033            [1353]    Sheridan    Wyoming   
183       56037            [1317]  Sweetwater    Wyoming   
184       56039             [188]       Teton    Wyoming   

                      pop_li  \
0                    [19000]   
1                    [72000]   
2                    [11000]   
3             [10000, 20000]   
4                     [3500]   
..                       ...   
180  [33000, 360000, 345000]   
181                

In [5]:
#'wwtp_id','county_fips','county','state','population_served','ww_date','flow_rate','per_capita_flow_rate','sample_location','pcr_conc_gc/cap/d','normalization_method','sampling_week','pcr_conc_gc/mL','case_date','cases_per_100k','cases'
count = 0
for wwtp_id in wwtp_id_li:
    county_fips,county,state,pop,ww_date,flow_rate,per_capita_flow_rate,sample_location,gc_cap_d,n_method,sampling_week,gc_ml,case_date,cases_per_100k,cases = \
        None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
    for ind,val in county_df['wwtp_id'].items():
        if wwtp_id in val:
            county_fips, county, state = county_df.loc[ind,['county_fips','county','state']]
            sample_location_li = eval(county_df.loc[ind,'sample_location_li'])
            sample_location = sample_location_li[val.index(wwtp_id)]   
            pop_li = eval(county_df.loc[ind,'pop_li'])
            pop = pop_li[val.index(wwtp_id)]
            pop_frac_arr = np.array(pop_li)/sum(pop_li)
            pop_frac = pop_frac_arr[val.index(wwtp_id)]        
            break
    temp_wc_df = wc_df[wc_df['fipscode'] == int(county_fips)]
    temp_cc_df = cc_df[cc_df['fipscode'] == int(county_fips)].sort_values(by='date')
    wwtp_id = str(wwtp_id)
    for ind1,val1 in cd_df['key_plot_id'].items():
        ww_date,gc_cap_d,n_method,case_date,cases_per_100k,cases = None,None,None,None,None,None
        if val1.split('_')[0] == 'CDC':
            wwtp_id_to_match = val1.split('_')[3]
        elif val1.split('_')[0] in ['NWSS','WWS']:
            wwtp_id_to_match = val1.split('_')[2]
        if (wwtp_id == wwtp_id_to_match) & (~np.isnan(cd_df.loc[ind1,'pcr_conc_smoothed'])):
            ww_date,gc_cap_d,n_method = cd_df.loc[ind1,['date','pcr_conc_smoothed','normalization']]
            for ind2,val2 in temp_wc_df['sampling_week'].items():
                sampling_week,gc_ml,flow_rate,per_capita_flow_rate = None,None,None,None
                if ww_date.isocalendar().week == val2.isocalendar().week:
                    sampling_week = val2
                    gc_ml = temp_wc_df.loc[ind2,'effective_concentration_rolling_average']
                    flow_rate = gc_cap_d*pop/gc_ml/1000
                    per_capita_flow_rate = flow_rate/pop
                    break
            date_diffs = abs(ww_date - temp_cc_df['date'])
            case_date, cases_per_100k = temp_cc_df.iloc[date_diffs.argmin()][['date','rolling_average_cases_per_100k_centered']]
            cases = cases_per_100k*pop_frac*pop/100_000   
            row_to_append = pd.DataFrame({'wwtp_id':[wwtp_id], 'county_fips':[county_fips], 'county':[county], 'state':[state], 'population_served':[pop], \
                                'ww_date':[ww_date], 'flow_rate':[flow_rate], 'per_capita_flow_rate':[per_capita_flow_rate], 'sample_location':[sample_location], \
                                'pcr_conc_gc/cap/d':[gc_cap_d], 'normalization_method':[n_method], 'sampling_week':[sampling_week], \
                                'pcr_conc_gc/mL':[gc_ml], 'case_date':[case_date], 'cases_per_100k':[cases_per_100k], 'cases':[cases]}) 
            df = pd.concat([df,row_to_append])
    count += 1
    print(count)
        
print(wwtp_id,county_fips,county,state,pop,ww_date,flow_rate,per_capita_flow_rate,sample_location,gc_cap_d,n_method,sampling_week,gc_ml,case_date,cases_per_100k,cases)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: Runtime

47


/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: Runtime

48


...

328


/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: Runtime

329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356


/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  flow_rate = gc_cap_d*pop/gc_ml/1000
/var/folders/tm/lml2dr_d2y14c8k5z7_kvnvc0000gn/T/ipykernel_77739/4237109223.py:32: Runtime

357
358
359
360
361
362
363
364
188 56039 Teton Wyoming 13036 None 24233763.86527309 1858.9877159614214 Treatment plant None None 2022-12-14 00:00:00 349.6252772017715 None None None


In [7]:
df.to_csv('compiled_data.csv',index=False)


In [5]:
county_df.to_csv('county_df.csv',index=False)

In [7]:
county_df = pd.read_csv('county_df.csv')
county_df

,county_fips,wwtp_id,county,state,pop_li,sample_location_li
0,1033,[1333],Colbert,Alabama,[19000],['Treatment plant']
1,1081,[1464],Lee,Alabama,[72000],['Treatment plant']
2,1117,[1394],Shelby,Alabama,[11000],['Treatment plant']
3,2110,"[1490, 1491]",Juneau,Alaska,"[10000, 20000]","['Treatment plant', 'Treatment plant']"
4,4012,[501],La Paz,Arizona,[3500],['Treatment plant']
...,...,...,...,...,...,...
180,55025,"[391, 1277, 231]",Dane,Wisconsin,"[33000, 360000, 345000]","['Treatment plant', 'Treatment plant', 'Treatm..."
181,55073,[433],Marathon,Wisconsin,[42350],['Treatment plant']
182,56033,[1353],Sheridan,Wyoming,[20000],['Treatment plant']
183,56037,[1317],Sweetwater,Wyoming,[12000],['Treatment plant']
